In [36]:
import pandas as pd

from DataPreprocessing import *

In [37]:
df = pd.read_csv("./data/3_test.csv")
df

,ID,area,ck,ck-saveall,ck-drawall,ck-savetime,ck-drawtime,ck-saveavg,ck-drawavg,ck-avg,...,dep-avg,dep-9201,fed-9201,fed-avg,comp,ck-changame,dep-changame,ck-changtime,dep-changtime,VV
0,1,104,?,0,0,0,0,?,NaN,0,...,0,NaN,0,0,.,0,0,.,0,Withheld
1,2,111,?,0,?,0,0,NaN,0,0,...,0,0,?,0,?,0,0,0,.,Withheld
2,8,600,0,?,0,NaN,0,?,0,NaN,...,.,?,NaN,0,0,0,0,?,?,Withheld
3,12,?,0,0,0,?,0,0,0,0,...,0,0,0,?,0,?,0,0,0,Withheld
4,13,?,0,0,0,0,0,0,0,0,...,130,0,?,0,0,0,0,0,0,Withheld
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6532,32656,30000,0,0,?,0,0,.,0,0,...,408,0,.,0,NaN,NaN,0,0,0,Withheld
6533,32662,?,0,0,0,NaN,0,0,0,0,...,18,0,0,?,0,0,0,?,0,Withheld
6534,32664,NaN,?,0,0,0,0,0,0,0,...,78933,NaN,?,0,?,0,11250,NaN,2,Withheld
6535,32669,.,NaN,0,0,0,0,?,0,0,...,.,0,0,0,.,0,0,0,0,Withheld


In [38]:
dp = DataPreprocessing()
dp.load_data("./data/3_train.csv", './data/3_test.csv', index_col='ID')

加载数据
---------------------------------
如果报错，请检查文件相对路径:
/Users/proffl/TwilightStruggle/MachineLearning/practice/./data/3_train.csv
合计：26144行， 25列
前五行数据为（如果显示乱码，用dp.load_data(..., encoding='utf-8')等重新尝试：
   area   ck ck-saveall ck-drawall ck-savetime ck-drawtime ck-saveavg ck-drawavg ck-avg dep-saveall  ... dep-avg dep-9201 fed-9201 fed-avg comp ck-changame dep-changame ck-changtime dep-changtime VV
ID                                                                                                   ...                                                                                              
3   709    0          ?        NaN           ?           ?          0        NaN      ?           0  ...       0        0      NaN       0    0           0            0            0             0  0
4     .  NaN          0          0           0         NaN          0          0      ?           0  ...       0        ?        ?       0    0           0            0            0             0  0
5 

(26144, 25)

In [39]:
def area_handler(area):
    str_area = str(area).strip()
    if str_area in ['.', '?', 'nan']:
        return str("0")
    if len(str_area) == 0:
        return str("0")
    return int(str_area[:1]) # int(), float()


data_area = dp.data['area'].apply(lambda area: area_handler(str(area))).astype(str)
data_area1 = data_area.apply(lambda area: area[:1])
data_area1 = pd.get_dummies(data_area1, prefix='area')
data_area1


,area_0,area_1,area_2,area_3,area_4,area_5,area_6,area_7,area_8,area_9
ID,,,,,,,,,,
3,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,1,0,0,0
7,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
32678,0,1,0,0,0,0,0,0,0,0
32679,0,0,0,0,0,0,0,0,1,0
32680,0,1,0,0,0,0,0,0,0,0


In [40]:
def numeric_to_int(x, na_val=-1):
    import sys
    if x is None:
        return na_val
    if type(x) == str:
        if x in ['?', '.', 'NaN']:
            return na_val
        try:
            result = int(x)
        except ValueError:
            result = sys.maxsize
        return result
    if np.isnan(x):
        return na_val
    return min(x, sys.maxsize)

def multi_fit(x, y, z):
    newx = numeric_to_int(x)
    newy = numeric_to_int(y)
    newz = numeric_to_int(z)
    if newx == -1:
        if newy != -1 and newz != -1:
            newx = newy * newz
    if newy == -1:
        if newx != -1 and newz > 0:
            newy = newx / newz
    if newz == -1:
        if newx != -1 and newy > 0:
            newz = newx / newy
    if (newx == -1 or newx == 0) and (newy == -1 or newy == 0) and (newz == -1 or newz == 0):
        newx = newy = newz = 0

    return newx, newy, newz
data_ck_saveall, data_ck_savetime, data_ck_saveavg = \
    zip(*dp.data.apply(lambda row: multi_fit(row['ck-saveall'], row['ck-savetime'], row['ck-saveavg']), axis=1))
data_ckall = pd.concat([pd.Series(data_ck_saveall), pd.Series(data_ck_savetime), pd.Series(data_ck_saveavg)], axis=1)
data_ckall.set_index(dp.data.index)

data_ckall

,0,1,2
0,0,0.00,0.00
1,0,0.00,0.00
2,0,0.00,0.00
3,0,0.00,0.00
4,0,0.00,0.00
...,...,...,...
26139,0,0.00,0.00
26140,0,0.00,0.00
26141,0,0.00,0.00
26142,0,0.00,0.00


In [41]:
from sklearn.impute import KNNImputer
ck_knn_im = KNNImputer(missing_values=-1, n_neighbors=10)
data_ckall = ck_knn_im.fit_transform(data_ckall)
data_ckall = pd.DataFrame(data_ckall, columns=['ck-saveall', 'ck-savetime', 'ck-saveavg'], index=dp.data.index)
data_ckall[data_ckall['ck-saveall']>10000]


,ck-saveall,ck-savetime,ck-saveavg
ID,,,
21,467496.00,18.00,25972.00
90,100160.00,9.00,11129.00
195,12181361.00,107.00,113844.50
205,2252177.00,34.00,66240.50
207,1390844.00,28.00,49673.00
...,...,...,...
32608,70000.00,1.00,70000.00
32629,950128.00,19.00,50007.00
32647,182369.00,7.00,49808.40


In [42]:
data_ck_drawall, data_ck_drawtime, data_ck_drawavg = \
    zip(*dp.data.apply(lambda row: multi_fit(row['ck-drawall'], row['ck-drawtime'], row['ck-drawavg']), axis=1))
data_drawall = pd.concat([pd.Series(data_ck_drawall), pd.Series(data_ck_drawtime), pd.Series(data_ck_drawavg)], axis=1)
data_drawall.set_index(dp.data.index)

,0,1,2
ID,,,
3,0,0.00,0.00
4,0,0.00,0.00
5,0,0.00,0.00
6,0,0.00,0.00
7,0,0.00,0.00
...,...,...,...
32678,0,0.00,0.00
32679,0,0.00,0.00
32680,0,0.00,0.00


In [43]:
dk_knn_im = KNNImputer(missing_values=-1, n_neighbors=10)
data_drawall = dk_knn_im.fit_transform(data_drawall)
data_drawall = pd.DataFrame(data_drawall, columns=['dk-drawall', 'dk-drawtime', 'dk-drawavg'], index=dp.data.index)
data_drawall[data_drawall['dk-drawall']>0]

,dk-drawall,dk-drawtime,dk-drawavg
ID,,,
21,467109.00,17.00,27477.00
195,8483977.00,162.00,52370.00
205,7015.00,1.00,7015.00
207,98526.00,2.00,49263.00
208,113684621.00,27.00,4210542.00
...,...,...,...
32598,307294.00,4.00,76824.00
32608,2589619.90,55.00,47133.00
32629,232806.00,6.00,38801.00


In [44]:
data_dep_saveall, data_dep_savetime, data_dep_saveavg = \
    zip(*dp.data.apply(lambda row: multi_fit(row['dep-saveall'], row['dep-savetime'], row['depsaveavg']), axis=1))
data_depall = pd.concat([pd.Series(data_dep_saveall), pd.Series(data_dep_savetime), pd.Series(data_dep_saveavg)], axis=1)
data_depall.set_index(dp.data.index)


dep_knn_im = KNNImputer(missing_values=-1, n_neighbors=10)
data_depall = dep_knn_im.fit_transform(data_depall)
data_depall = pd.DataFrame(data_depall, columns=['dep-saveall', 'dep-savetime', 'depsaveavg'], index=dp.data.index)
data_depall[data_depall['dep-saveall']>0]

,dep-saveall,dep-savetime,depsaveavg
ID,,,
137,3185841.00,27.00,117994.00
152,129609.00,1.00,129609.00
158,400000.00,2.00,200000.00
163,284651.00,5.00,56930.20
165,1961800.00,54.50,96868.42
...,...,...,...
32671,50236758.00,319.00,157482.00
32674,1474469.00,10.00,147447.00
32675,236055.00,21.00,11241.00


In [45]:
data_dep_drawall, data_dep_drawtime, data_dep_drawavg = \
    zip(*dp.data.apply(lambda row: multi_fit(row['dep-drawall'], row['dep-drawtime'], row['depdrawavg']), axis=1))
data_depdrawall = pd.concat([pd.Series(data_dep_drawall), pd.Series(data_dep_drawtime), pd.Series(data_dep_drawavg)], axis=1)
data_depdrawall.set_index(dp.data.index)
dep_draw_knn_im = KNNImputer(missing_values=-1, n_neighbors=10)
data_depdrawall = dep_draw_knn_im.fit_transform(data_depdrawall)
data_depdrawall = pd.DataFrame(data_depdrawall, columns=['dep-drawall', 'dep-drawtime', 'depdrawavg'], index=dp.data.index)
data_depdrawall[data_depdrawall['dep-drawall']>0]


,dep-drawall,dep-drawtime,depdrawavg
ID,,,
137,2989500.00,22.00,135886.00
148,600000.00,1.60,343000.00
152,534559.00,3.00,178186.00
155,1200000.00,1.00,1200000.00
158,537000.00,7.00,76714.00
...,...,...,...
32671,52642773.00,78.00,674907.35
32674,664000.00,6.00,110666.67
32675,2411566.00,28.00,86127.00


In [46]:
data_ck_avg = dp.data['ck-avg'].apply(lambda x: numeric_to_int(x))
data_dep_avg = dp.data['dep-avg'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_dep9201 = dp.data['dep-9201'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_fed9201 = dp.data['fed-9201'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_fed_avg = dp.data['fed-avg'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_ck_changame = dp.data['ck-changame'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_dep_changame = dp.data['dep-changame'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_ck_changtime = dp.data['ck-changtime'].apply(lambda x: numeric_to_int(x, na_val=-1))
data_dep_changtime = dp.data['dep-changtime'].apply(lambda x: numeric_to_int(x, na_val=-1))

In [47]:
comp_mapping = {'0': 0, '?': 2, '1': 1, '.': 2}
data_comp = dp.data['comp'].map(comp_mapping)
data_comp = pd.get_dummies(data_comp)
data_comp

,0.00,1.00,2.00
ID,,,
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,0,0,1
...,...,...,...
32678,1,0,0
32679,1,0,0
32680,1,0,0


In [48]:
data_all = pd.concat([data_area1, data_ckall, data_drawall, data_ck_avg, data_dep_avg,
                      data_depall, data_depdrawall,
                      data_dep9201, data_fed9201, data_fed_avg, data_ck_changame, data_dep_changame,
                      data_ck_changtime, data_dep_changtime], axis=1)

In [49]:
from sklearn.impute import SimpleImputer
all_knn_imp = SimpleImputer(strategy="most_frequent")
columns = data_all.columns
data_all = all_knn_imp.fit_transform(data_all)
data_all = pd.DataFrame(data_all, columns=columns, index=dp.data.index)
data_all

,area_0,area_1,area_2,area_3,area_4,area_5,area_6,area_7,area_8,area_9,...,dep-drawall,dep-drawtime,depdrawavg,dep-9201,fed-9201,fed-avg,ck-changame,dep-changame,ck-changtime,dep-changtime
ID,,,,,,,,,,,,,,,,,,,,,
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,-1.00,0.00,0.00,0.00,0.00,0.00
4,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,-1.00,-1.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,-1.00,0.00,-1.00,0.00,0.00,0.00,-1.00
6,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,-1.00,0.00,0.00,0.00,0.00
7,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32678,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,-1.00,-1.00,-1.00,0.00,0.00,-1.00
32679,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,...,922337203685506688.00,4972554392.89,3379.00,0.00,0.00,-1.00,0.00,-1.00,0.00,0.00
32680,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [50]:
data_all = pd.concat([df['ID'], data_all, data_comp, dp.data['VV']], axis=1)
data_all.to_csv("./data/features3_test_2.csv", index=False)
